<a href="https://colab.research.google.com/github/Gapilesh/BlinkKey_FinalProject/blob/main/Eyeblink8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Mount Goggle Drive

In [2]:
from google.colab import drive

In [3]:
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Step 2: Unzip the dataset

In [4]:
import zipfile

# Path to the zip file
zip_path = '/content/drive/MyDrive/Dataset/Eyeblink/eyeblink8.zip'

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/eyeblink8')

print("Dataset unzipped successfully!")

Dataset unzipped successfully!


Step 3: Explore the Dataset

In [5]:
import os

# Path to the unzipped dataset
dataset_path = '/content/eyeblink8/eyeblink8'

# List folders (1, 2, 3, 4, 8, 9, 10, 11)
folders = os.listdir(dataset_path)
print("Folders in the dataset:", folders)

# Check contents of one folder (e.g., folder '1')
sample_folder = os.path.join(dataset_path, '1')
sample_files = os.listdir(sample_folder)
print("Files in folder '1':", sample_files)

Folders in the dataset: ['8', '9', '2', '10', '1', '4', '11', '3']
Files in folder '1': ['26122013_223310_cam.tag', '26122013_223310_cam.txt', '26122013_223310_cam.avi']


Step 4: Load and Preprocess the Data

In [6]:
!pip install opencv-python

In [8]:
import cv2
import os

# Function to extract frames from a video
def extract_frames(video_path, output_folder, frame_limit=1000):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open the video
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret or frame_count >= frame_limit:
            break

        # Save the frame as an image
        frame_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames from {video_path}")

# Example: Extract frames from folder '1'
video_path = os.path.join(dataset_path, '1', '26122013_223310_cam.avi')  # Adjust based on your dataset
output_folder = '/content/eyeblink8/frames/1'
extract_frames(video_path, output_folder)


Extracted 1000 frames from /content/eyeblink8/eyeblink8/1/26122013_223310_cam.avi


Step 5: Parse annotation

In [9]:
# Function to parse annotations
def parse_annotations(annotation_path):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    # Process lines to extract blink labels
    blink_labels = []
    for line in lines:
        # Adjust this based on the format of your annotation file
        frame_number, label = line.strip().split()
        blink_labels.append(float(label))

    return blink_labels

# Example: Parse annotations for folder '1'
annotation_path = os.path.join(dataset_path, '1', '26122013_223310_cam.txt')  # Adjust based on your dataset
blink_labels = parse_annotations(annotation_path)
print("Blink labels for folder '1':", blink_labels[:10])  # Print first 10 labels

Blink labels for folder '1': [0.142686, 0.200098, 0.222628, 0.254634, 0.297089, 0.31903, 0.350769, 0.382737, 0.415133, 0.464508]


Step 6: Build the Model

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, TimeDistributed, Flatten

# Define the model
model = Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(None, 224, 224, 3)),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(64, (3, 3), activation='relu')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Flatten()),
    LSTM(64, return_sequences=True),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, None, 222, 222, 32)  │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, None, 111, 111, 32)  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, None, 109, 109, 64)  │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, None, 54, 54, 64)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, None, 186624)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, None, 64)            │      47,792,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, None, 1)             │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 47,811,841 (182.39 MB)

 Trainable params: 47,811,841 (182.39 MB)

 Non-trainable params: 0 (0.00 B)